###READ DATABASE

# todo:
-Equilibrer le dataset: TRIER LES POSITIFS QUI PRETENT A CONFUSION, en créer de nouveaux sur l'app, supprimer aléatoirement des négatifs, concat les datasets 1h FAIT
-nettoyer les phrase (stop words, ponctuation, lemmatisation) SI POSSIBLE
-tester des modèles: suivre le tuto
-renvoyer les résultats sur l'application (probabilité de positif si possible) """- Faire une interface permettant au client de voir les commentaires après classification""" Idealement possibilité de rentrer une phrase et d'avoir une probabilité de positif
-rendre le site plus joli + relecture

In [1]:
#TRAIN TEST SCRIPT
import sqlite3
import pandas as pd
import numpy as np
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French

db = sqlite3.connect('Application/database.db')
user_data = pd.read_sql_query("SELECT * FROM User_data", db)
scrapped_data = pd.read_sql_query("SELECT * FROM Posts WHERE label != 0 AND label != 'inexploitable' AND label != 'neutre'", db)
del scrapped_data['topic']
df = pd.concat([scrapped_data, user_data])
df = df.reset_index()
del df['index']

positives = df[df['label'] == 'insultant']
negatives = df[df['label'] == 'pas insultant']
negatives_sample = negatives.sample(n = len(positives)) #on équilibre le nb de positifs et de négatifs
df = pd.concat([negatives_sample, positives])
df = df.reset_index()
del df['index']

#encode labels
df = df.replace('pas insultant', 0)
df = df.replace('insultant', 1)
corpus = df['message']

#train_test_split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

#découpage data/label
y_train = train['label']
y_test = test['label']
x_train = train['message']
x_test = test['message']
x_test

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
vocab = vec.fit(corpus)
x_train_tfidf = vocab.transform(x_train)
x_test_tfidf = vocab.transform(x_test)
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB().fit(x_train_tfidf, y_train)

from sklearn.linear_model import SGDClassifier
SGD_clf = SGDClassifier().fit(x_train_tfidf, y_train)

predicted_NB = NB_clf.predict(x_test_tfidf)
print(np.mean(predicted_NB == y_test))
predicted_SGD = SGD_clf.predict(x_test_tfidf)
print(np.mean(predicted_SGD == y_test))

(102, 801)
(26, 801)
0.6923076923076923
0.6923076923076923


In [2]:
import sqlite3
import pandas as pd
import numpy as np
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French

In [3]:
db = sqlite3.connect('Application/database.db')
user_data = pd.read_sql_query("SELECT * FROM User_data", db)
scrapped_data = pd.read_sql_query("SELECT * FROM Posts WHERE label != 0 AND label != 'inexploitable' AND label != 'neutre'", db)
del scrapped_data['topic']
df = pd.concat([scrapped_data, user_data])
df = df.reset_index()
del df['index']
df

,message,label
0,C'est un tireur qui prend en otage deux personnes,pas insultant
1,\nFolaillon,pas insultant
2,Aucune attaque contre les catholiques circulez,pas insultant
3,"\nOk, lourdes c'est pas en France.",pas insultant
4,C'est pas un risque d'attentat islamique donc ...,pas insultant
5,Pareil j'ai lu Londres,pas insultant
6,\nOuais j'ai fait le topic avant de lire l'art...,pas insultant
7,il disait bonjour,pas insultant
8,\nNon mais Lourdes est connu mondialement ...,pas insultant
9,\nMes parents habitent à 35/40 km de Lourdes a...,pas insultant


In [5]:
positives = df[df['label'] == 'insultant']
negatives = df[df['label'] == 'pas insultant']
negatives_sample = negatives.sample(n = len(positives)) #on équilibre le nb de positifs et de négatifs
df = pd.concat([negatives_sample, positives])
df = df.reset_index()
del df['index']
df

,message,label
0,La capote elle est ou ??,pas insultant
1,impressionnant,pas insultant
2,\nPremier nutritionniste diplômé\nLaisse-moi d...,pas insultant
3,"\nC'est Affairiste, désolé.",pas insultant
4,Je fav bon courage pour ta reconstruction.,pas insultant
5,Bordel en plus ils me demande dix milles captc...,pas insultant
6,"Assumez hein, on vous voit.",pas insultant
7,\nHonnetement Dans tout les cas Je pense qu'il...,pas insultant
8,on peut faire le test rapidement ? ou ?,pas insultant
9,Trine 3 bordel l’histoire commence quand le je...,pas insultant


# STEMMING REMOVE STOP WORDS AND PUNCTUATION

In [6]:
import spacy
from nltk.stem.snowball import SnowballStemmer
nlp = spacy.load('fr')

doc = nlp("bonjour, j'aime énormément les animaux de compagnie")

#for token in doc:
#    print(token, token.lemma, token.lemma_)

tokens = [str(word) for word in doc if word.is_stop == False and not word.is_punct and word.lemma_ != '\n']
print(tokens)
#Str = ' '.join(lem)
#print(Str)


stemmer = SnowballStemmer(language='french')
stemmatized = []
for token in tokens:  
    stemmatized.append(stemmer.stem(token))
print(stemmatized)
final = ' '.join(stemmatized)
final

['bonjour', 'aime', 'énormément', 'animaux', 'compagnie']
['bonjour', 'aim', 'énorm', 'animal', 'compagn']


'bonjour aim énorm animal compagn'

In [5]:
import spacy
from nltk.stem.snowball import SnowballStemmer
def treat_sentence(string, nlp = spacy.load('fr'), stemmer = SnowballStemmer(language='french')):
    #nlp = spacy.load('fr')
    doc = nlp(string)
    tokens = [str(word) for word in doc if not word.is_punct and word.lemma_ != '\n'] #word.is_stop == False and  (pour supprimer les stops words)
    #stemmer = SnowballStemmer(language='french')
    stemmatized = []
    for token in tokens:  
        stemmatized.append(stemmer.stem(token))
    final = ' '.join(stemmatized)
    return final
treat_sentence("bonjour, j'aime énormément les animaux de compagnie")

"bonjour j' aim énorm le animal de compagn"

In [13]:
for sentence in df['message']:
    print(sentence)
    print(treat_sentence(sentence))
    sentence = treat_sentence(sentence)
df

La capote elle est ou ??
la capot elle est ou
impressionnant
impression

Premier nutritionniste diplômé
Laisse-moi deviner, tu pratiques un vrai sport, tu es juste tracé et ça te suffit largement. Tu baises. C’est ça ?
Sinon bon boulot l’auteur.
premi nutrition diplôm laiss moi devin tu pratiqu un vrai sport tu e just trac et ça te suff larg tu bais c’ est ça sinon bon boulot l’ auteur

C'est Affairiste, désolé.
c' est affair désol
Je fav bon courage pour ta reconstruction.
je fav bon courag pour ta reconstruct
Bordel en plus ils me demande dix milles captcha là
bordel en plus il me demand dix mill captch là
Assumez hein, on vous voit.
assum hein on vous voit

Honnetement Dans tout les cas Je pense qu'il y aurait eu une reconquista franque donc Je sais pas
honnet dan tout le cas je pens qu' il y aur eu une reconquist franqu donc je sais pas
on peut faire le test rapidement ? ou ?
on peut fair le test rapid ou
Trine 3 bordel l’histoire commence quand le jeux s’arrete
trin 3 bordel l’ hi

tu est tel con que je c mem pas poirquoi dam natur ta mis au mond elle était bour quand elle ta cré je pans
T qu'un ptit branleur pervers des vitrines de bruxelle 
t qu' un ptit branleur perver de vitrin de bruxel
Hersat d'excrément putréfié
hersat d' excrément putréfi
Ton avis m'est aussi précieux qu'un trou de balle au niveau du coude.
ton avis m' est auss précieux qu' un trou de ball au niveau du coud
Tu as réussi à trouver ta bite avant de te branler? T'a utilisé un microscope?
tu as réuss à trouv ta bit avant de te branl t' a utilis un microscop
Ton père aurait dû utiliser des mouchoirs à la place d'infecter ta mère la pute.
ton per aur dû utilis de mouchoir à la plac d' infect ta mer la put
Ta bete gueule ressamble a un cul de babouin qui vien de se fair sodomisé par un elephant
ta bet gueul ressambl a un cul de babouin qui vien de se fair sodomis par un eleph
Les grosse truie dans ton genre son rare, rarement aussi horrible
le gross trui dan ton genr son rar rar auss horribl
vie

,message,label
0,La capote elle est ou ??,pas insultant
1,impressionnant,pas insultant
2,\nPremier nutritionniste diplômé\nLaisse-moi d...,pas insultant
3,"\nC'est Affairiste, désolé.",pas insultant
4,Je fav bon courage pour ta reconstruction.,pas insultant
5,Bordel en plus ils me demande dix milles captc...,pas insultant
6,"Assumez hein, on vous voit.",pas insultant
7,\nHonnetement Dans tout les cas Je pense qu'il...,pas insultant
8,on peut faire le test rapidement ? ou ?,pas insultant
9,Trine 3 bordel l’histoire commence quand le je...,pas insultant


# PREPROCESSING

In [230]:
#encode labels
df = df.replace('pas insultant', 0)
df = df.replace('insultant', 1)
corpus = df['message']

In [231]:
#train_test_split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [232]:
#découpage data/label
y_train = train['label']
y_test = test['label']
x_train = train['message']
x_test = test['message']
x_test

0                                    \nJe ne mérite rien
36     \nNon plus\nComme a dit Staline: "Là où il y'a...
32     Pareil,\nChristine & the queen, niki minaj, ka...
27                                               Profite
17     \nAh ok je pensais que le brevet était indispe...
37     \nHonnetement Dans tout les cas Je pense qu'il...
46                                 \ncette mauvaise fois
5                                     Post dans le doute
63     Faut vraiment être un singe pour ne pas savoir...
59                    \nRegarde ma signature sac à merde
69                      Putain mais quel cuck défonce le
108                                   vieux sac a sperme
19                                  Onch c'est drôle ~ ~
43                                    \nQui va feed ça ?
106    Ta bete gueule ressamble a un cul de babouin q...
73                   Qu’est ce que c’est que ce laideron
42     En vrai, je peux le tenter et m’en prendre cin...
87     Ah bah merde, t'essais p

In [238]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
vocab = vec.fit(corpus)
x_train_tfidf = vocab.transform(x_train)
x_test_tfidf = vocab.transform(x_test)
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

(91, 781)
(23, 781)


'(?u)\\b\\w\\w+\\b'

# MODELES

In [234]:
from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB().fit(x_train_tfidf, y_train)

In [235]:
from sklearn.linear_model import SGDClassifier
SGD_clf = SGDClassifier().fit(x_train_tfidf, y_train)

C:\Users\grego\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [236]:
predicted_NB = NB_clf.predict(x_test_tfidf)
print(np.mean(predicted_NB == y_test))
predicted_SGD = SGD_clf.predict(x_test_tfidf)
print(np.mean(predicted_SGD == y_test))

0.6086956521739131
0.782608695652174


https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

#TESTS

In [81]:
doc = nlp(labellized_data['message'].iloc[100])
doc.text
lem = [word.lemma_ for word in doc if word.is_stop == False and not word.is_punct and word.lemma_ != '\n']

"Après ce saut dans la flotte m'a montré que je savais nager, tant pis si ça ne marche pas pour toi l'auteur"

In [ ]:
lem = [word.lemma_ for word in doc if word.is_stop == False and not word.is_punct and word.lemma_ != '\n']